## 不同的Memory工具

- 短时记忆：存储在内存中
- 构建记忆实体清单
- 接入知识图谱
- 长对话在内存中的处理方式
- 长时记忆实现方式

### 内存中的短时记忆

In [2]:
from langchain.memory import  ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("你好，我是人类！")
memory.chat_memory.add_ai_message("你好，我是AI，有什么可以帮助你的吗？")

memory.load_memory_variables({})

C:\Users\Tingwang.Gao\AppData\Local\Temp\ipykernel_31620\4077081215.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': 'Human: 你好，我是人类！\nAI: 你好，我是AI，有什么可以帮助你的吗？'}

In [3]:
#实现一个最近的对话窗口，超过窗口条数的对话将被删除
from langchain.memory import  ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1) # 只保留一句对话

memory.save_context({"input":"你好，我是人类!"},{"output":"你好，我是AI，有什么可以帮助你的吗？"})
memory.save_context({"input":"我想吃鸡肉"},{"output":"好的，我帮你找找鸡肉的做法"})

memory.load_memory_variables({})

C:\Users\Tingwang.Gao\AppData\Local\Temp\ipykernel_31620\4185949352.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


{'history': 'Human: 我想吃鸡肉\nAI: 好的，我帮你找找鸡肉的做法'}

### 记忆实体清单

In [ ]:
# 构建记忆实体概念清单
from langchain.llms import  OpenAI
from langchain.memory import ConversationEntityMemory

llm = OpenAI(
    temperature=0
)

memory = ConversationEntityMemory(llm=llm)
_input = {
    "input":"胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角."
}
# memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {
        "output":"听起来很刺激，我也想加入他们！"
    }
)
memory.load_memory_variables({"input":"铁三角是谁?"})

### 知识图谱

In [ ]:
from langchain.llms import  OpenAI
from langchain.memory import ConversationKGMemory

llm = OpenAI(
    temperature=0
)   

memory = ConversationKGMemory(llm=llm,return_messages=True)

memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)

memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)

memory.load_memory_variables({"input":"tomie是谁?"})

## 长对话在内存中的处理方式
- 总结摘要，减少存储内容
- token计算

In [ ]:
# 总结摘要
from langchain.memory import ConversationSummaryMemory
from langchain.llms import  OpenAI

llm = OpenAI(
    temperature=0
)

memory = ConversationSummaryMemory(llm=llm)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)

memory.load_memory_variables({})
# 产生摘要
memory.predict_new_summary(messages,"")

In [ ]:
#使用ChatMessageHistory来快速获得对话摘要
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ChatMessageHistory
from langchain.llms import  OpenAI

hisiory = ChatMessageHistory()
hisiory.add_user_message("你好，我是人类！")
hisiory.add_ai_message("你好，我是AI小丸子，有什么可以帮助你的吗？")

#memory = ConversationSummaryMemory.from_messages(
#    llm=OpenAI(temperature=0),
#    chat_memory=hisiory,
#    return_messages=True
#)
# memory.buffer

memory = ConversationSummaryMemory(
    llm=OpenAI(temperature=0),
    return_messages=True,
    buffer="The AI introduces itself as AI Little Maruko and asks if there is anything it can help the human with.",
    chat_memory=hisiory
)
memory.buffer

In [ ]:
# 当对话持续进行且对话内容很多的时候
# 可以使用ConversationSummaryBufferMemory来存储对话摘要
# 这是一种非常有用的方式,它会根据token的数量来自动判断是否需要进行摘要
# 当token数量超过阈值的时候,会自动进行摘要
# 在缓冲区中,会保留最近的k条对话
# 比较久的对话会被删除，在删除前会进行摘要

from langchain.memory import ConversationSummaryBufferMemory
from langchain.llms import OpenAI
llm = OpenAI(temperature=0)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150
)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了。"}
)

In [ ]:
# Conversation Token Buffer使用token长度来决定什么时候刷新内存

from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = OpenAI(temperature=0)
memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=150
)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了。"}
)

## 长时记忆的实现方式
通过向量数据库来存储之前的对话内容，有的向量数据库服务还提供自动摘要等，每次对话的时候，都会从向量数据库里查询最相关的文档或历史对话

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

memory = ConversationBufferMemory()
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了。"}
)

vectorstore = FAISS.from_texts(
    memory.buffer.split("\n"),
    OpenAIEmbeddings()
)
FAISS.save_local(vectorstore,"test_faiss")

In [ ]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory
r1 = FAISS.load_local("test_faiss",OpenAIEmbeddings(),allow_dangerous_deserialization=True)
r2 = r1.as_retriever(
    search_kwargs={"k":1}
)
memory2 = VectorStoreRetrieverMemory(
    retriever=r2
)
memory2.load_memory_variables({"prompt":"tomie是什么职业?"})